## Convolution 1-D for RMM2 network for GPU tuning 

Written by Abirlal Metya, Panini Dasgupta, Manmeet Singh (16/01/2020)

import modules

In [1]:
import numpy as np
from numpy.random import seed
seed(7)
import random as rn
rn.seed(7)
import tensorflow
tensorflow.random.set_seed(10)
import os
os.environ["PYTHONHASHSEED"] = '0'

import pandas as pd
import datetime
import hilbert_data1_jgrjd_20CRV3
from sklearn.preprocessing import MinMaxScaler
import itertools
import multiprocessing
from IPython.display import clear_output
import tqdm

import keras 
from keras.models import Sequential
from keras.layers import Input,Dense, Conv1D, Flatten,MaxPooling1D,Dropout, Activation, Flatten,Add
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


### Test and Train Splitter:

#### RMM2

In [2]:
x_train,_,y_train = hilbert_data1_jgrjd_20CRV3.data_hilbert(datetime.datetime(1979,1,1),datetime.datetime(2008,12,31))
x_test,_,y_test = hilbert_data1_jgrjd_20CRV3.data_hilbert(datetime.datetime(1974,6,1),datetime.datetime(1978,3,16))
x_test2,_,y_test2 = hilbert_data1_jgrjd_20CRV3.data_hilbert(datetime.datetime(2009,1,1),datetime.datetime(2015,12,31))


#### Historical pressure

In [3]:
x_test3 = hilbert_data1_jgrjd_20CRV3.data_pres(datetime.datetime(1905,1,1),datetime.datetime(2015,12,31))

#### scale the data

In [4]:
sc3 = MinMaxScaler()
sc5 = MinMaxScaler()

sc5.fit(x_test3[:])

test_x3 =  sc5.transform(x_test3[:])
train_x = sc5.transform(x_train[:])
test_x  = sc5.transform(x_test[:])
test_x2  = sc5.transform(x_test2[:])


sc3.fit(y_train[:])

train_y = sc3.transform(y_train)
test_y  = sc3.transform(y_test)
test_y2  = sc3.transform(y_test2)

#train_x.max(),test_x.max(),test_x3.max(),test_x2.max(),train_y.max(),test_y.max(),test_y2.max()

In RNN we have to choose a window. Here we choose first 120 points as predictor and next RMM value as predicted. That means RMM will be fitted using previous 120 time steps's pressure of every point

#### split the sequence data for training

In [5]:
def split_sequence(window,x,*args):
    xout  = []
    for i in range(window,len(x)):
        xout.append(x[i-window:i,:])
    
    xout = np.array(xout)
    xout = np.reshape(xout,(xout.shape[0],xout.shape[1],xout.shape[2]))
        
    if np.any(len(args)):
        for y in args:
            yout = []
            for i in range(window,len(y)):
                yout.append(y[i,0])
            yout = np.array(yout)
            yout = yout.reshape(yout.shape[0])
    else:
        yout = [] 
    
    return xout,yout

In [6]:
window = 120
xtrain , ytrain = split_sequence(window,train_x,train_y)
xtest , ytest   = split_sequence(window,test_x,test_y)
xtest2 , ytest2 = split_sequence(window,test_x2,test_y2)
xtest3,_        = split_sequence(window, test_x3)

#### Cut the data according to batch size

In [7]:
par_b =100 

#print(x_test3.shape)
te3_lc = ((len(x_test3)-window)//par_b)*par_b

xtest3 = xtest3[:te3_lc,:,:]
#print(xtest3.shape)

#x_test3.iloc[window:window+te3_lc,:].index
## THis perid data will be available



In [8]:
#print(xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)

tr_lc = ((len(x_train)-window)//par_b)*par_b
te_lc =  ((len(x_test)-window)//par_b)*par_b
te_lc2 =  ((len(x_test2)-window)//par_b)*par_b

xtrain = xtrain[:tr_lc,:,:]
ytrain = ytrain[:tr_lc]
xtest = xtest[:te_lc,:,:]
ytest = ytest[:te_lc,]
xtest2 = xtest2[:te_lc2,:,:]
ytest2 = ytest2[:te_lc2,]
#print(xtrain.shape,ytrain.shape,xtest.shape,xtest2.shape,ytest.shape,ytest2.shape)

### Using Simple Convolution 1D
* 1. Basic conv1d
* 2. wavenet
* 3. ENSO  paper model


In [4]:
nf=[]
for i in range(1,6,1):
    nf.append([2**i,2**i,2**i])
    nf.append([2**(i+2),2**(i+1),2**i])
    #nf.append([2**(2*i+1),2**(i+1),2**(i)])
    #nf.append([2**(i+1)+2**i,2**(i+1),2**(i-1)])

kz=[]
for i in range(1,4,1):
    kz.append([2**i,2**i,2**i])
    kz.append([2**(i+2),2**(i+1),2**(i)])
    #kz.append([2**(i+1)+2**i,2**(i+1),2**(i)]
    #kz.append([2**(i+1)+2**i,2**(i+1),2**(i-1)])

ds = [10,20,30,40,50]

In [10]:
def models(params):
    """
       Random number initializer is needed 
    """
    
    seed(7)
    rn.seed(7)
    tensorflow.random.set_seed(10)
    os.environ["PYTHONHASHSEED"] = '0'
    
    
    i = params[0]; k = params[1]; j = params[2];  
    #print('running on iteration ' + str(i)+','+str(k)+','+str(j))
    
    model = Sequential()
    # Use the Keras Conv1D function to create a 1-dimensional convolutional layer, with kernel size (filter) of 5X5 pixels and a stride of 1 in x and y directions. The Conv2D command automatically creates the activation function for you━here we use ReLu activation.

    model.add(Conv1D(nf[i][0] ,kernel_size=kz[k][0], strides=1,kernel_initializer = "random_uniform",
                     bias_initializer = "zeros",activation='relu',
                     input_shape=(xtrain.shape[1],xtrain.shape[2])))
    # Then use the MaxPooling2D function to add a 2D max pooling layer, with pooling filter sized 2X2 and stride of 2 in x and y directions.

    model.add(MaxPooling1D(pool_size=1, strides=1))

    
    model.add(Conv1D(nf[i][1], kernel_size=kz[k][1], strides=1,
                     kernel_initializer = "random_uniform",bias_initializer = "zeros",activation='relu'))
    # Then use the MaxPooling2D function to add a 2D max pooling layer, with pooling filter sized 2X2 and stride of 2 in x and y directions.

    model.add(MaxPooling1D(pool_size=1, strides=1))

    
    model.add(Conv1D(nf[i][2], kernel_size=kz[k][2], strides=1,
                     kernel_initializer = "random_uniform",bias_initializer = "zeros",activation='relu'))

    
    model.add(Flatten())
    model.add(Dense(ds[j], kernel_initializer = "random_uniform",
                    bias_initializer = "zeros",activation='relu'))


    model.add(Dense(1, activation='linear'))
    opt = keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
    model.compile(loss='mae', optimizer=opt)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,min_delta=.01,patience=30)

    
    model.fit(xtrain, ytrain, validation_data=(xtest, ytest),batch_size=100,callbacks=[es],shuffle= False, epochs= 200,verbose=0)
    
    train_corr_= np.nan
    test1_corr_ = np.nan
    test2_corr_ = np.nan
    
    np.seterr(divide='ignore', invalid='ignore')
    
    predict1   = model.predict(xtrain)
    yy_train   = sc3.inverse_transform(predict1)
    yy_train   = yy_train/yy_train.std()
    train_corr_ = np.corrcoef(yy_train[:,0],ytrain)[0,1]
    

    predict1  = model.predict(xtest2)
    yy_test1   = sc3.inverse_transform(predict1)
    yy_test1   = yy_test1/yy_test1.std()
    test1_corr_ = np.corrcoef(yy_test1[:,0],ytest2)[0,1]

    predict2  = model.predict(xtest)
    yy_test2   = sc3.inverse_transform(predict2)
    yy_test2   = yy_test2/yy_test2.std()
    test2_corr_ = np.corrcoef(yy_test2[:,0],ytest)[0,1]
    
    print('running on iteration ' + str(i)+','+str(k)+','+str(j))
    print(train_corr_,test1_corr_,test2_corr_)
    
    return i,k,j,train_corr_,test1_corr_,test2_corr_


In [ ]:
train_corr = np.zeros((len(nf),len(kz),len(ds)));train_corr.fill(-1)
test1_corr = np.zeros((len(nf),len(kz),len(ds)));test1_corr.fill(-1)
test2_corr = np.zeros((len(nf),len(kz),len(ds)));test2_corr.fill(-1)

f = open('/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/conv1d/RMM1_models.txt', 'w')

ii = range(len(nf))
kk = range(len(kz))
jj = range(len(ds))
          
paramlist = list(itertools.product(ii,kk,jj))
print(multiprocessing.cpu_count())
        
pool = multiprocessing.Pool(processes = 2 )

for i,k,j,train_corr_,test1_corr_,test2_corr_  in tqdm.tqdm(pool.imap_unordered(models, paramlist), total=len(paramlist)):
    train_corr[i,k,j] = train_corr_
    test1_corr[i,k,j] = test1_corr_
    test2_corr[i,k,j] = test2_corr_ 
  

28


  0%|          | 0/300 [00:00<?, ?it/s]

running on iteration 0,0,0
nan nan nan


  0%|          | 1/300 [00:26<2:12:57, 26.68s/it]

running on iteration 0,0,1
nan nan nan


...

 60%|█████▉    | 179/300 [1:24:21<1:36:57, 48.08s/it]

running on iteration 6,0,0
0.8218268135907718 0.8013622818235626 0.7349161304585416


 60%|██████    | 180/300 [1:25:10<1:36:41, 48.35s/it]

running on iteration 5,5,4
0.8496562113383156 0.7850022285595666 0.7379630423780646


 60%|██████    | 181/300 [1:25:26<1:17:03, 38.86s/it]

In [ ]:
np.save('train_rmm2_login.npy',train_corr)
np.save('test1_rmm2_login.npy',test1_corr)
np.save('test2_rmm2_login.npy',test2_corr)

In [2]:
train_corr = np.load('train_rmm2_login.npy')
test1_corr = np.load('test1_rmm2_login.npy')
test2_corr = np.load('test2_rmm2_login.npy')

In [6]:
te = test1_corr+test2_corr
i = np.where(te== np.nanmax(te))
train_corr[i],test1_corr[i],test2_corr[i]

(array([0.85598559]), array([0.8165648]), array([0.78725746]))

In [12]:
te = test1_corr+test2_corr
i = np.where(test2_corr== np.nanmax(test2_corr))
train_corr[i],test1_corr[i],test2_corr[i]

(array([0.83753546]), array([0.7816978]), array([0.79869823]))

In [14]:
nf,kz

([[2, 2, 2],
  [8, 4, 2],
  [4, 4, 4],
  [16, 8, 4],
  [8, 8, 8],
  [32, 16, 8],
  [16, 16, 16],
  [64, 32, 16],
  [32, 32, 32],
  [128, 64, 32]],
 [[2, 2, 2], [8, 4, 2], [4, 4, 4], [16, 8, 4], [8, 8, 8], [32, 16, 8]])

In [7]:
i

(array([7]), array([0]), array([0]))

In [8]:
nf[7]

[64, 32, 16]

In [9]:
kz[0]

[2, 2, 2]

In [10]:
ds[0]

10

In [15]:
nf=[]
for i in range(1,8,1):
    nf.append([2**i,2**i,2**i])
    nf.append([2**(i+2),2**(i+1),2**i])
    nf.append([2**(i+1)+2**i,2**(i+1),2**(i)])
    nf.append([2**(i+1)+2**i,2**(i+1),2**(i-1)])

kz=[]
for i in range(1,4,1):
    kz.append([2**i,2**i,2**i])
    kz.append([2**(i+2),2**(i+1),2**i])
    kz.append([2**(i+1)+2**i,2**(i+1),2**(i)])
    kz.append([2**(i+1)+2**i,2**(i+1),2**(i-1)])

In [17]:
nf,kz

([[2, 2, 2],
  [8, 4, 2],
  [6, 4, 2],
  [6, 4, 1],
  [4, 4, 4],
  [16, 8, 4],
  [12, 8, 4],
  [12, 8, 2],
  [8, 8, 8],
  [32, 16, 8],
  [24, 16, 8],
  [24, 16, 4],
  [16, 16, 16],
  [64, 32, 16],
  [48, 32, 16],
  [48, 32, 8],
  [32, 32, 32],
  [128, 64, 32],
  [96, 64, 32],
  [96, 64, 16],
  [64, 64, 64],
  [256, 128, 64],
  [192, 128, 64],
  [192, 128, 32],
  [128, 128, 128],
  [512, 256, 128],
  [384, 256, 128],
  [384, 256, 64]],
 [[2, 2, 2],
  [8, 4, 2],
  [6, 4, 2],
  [6, 4, 1],
  [4, 4, 4],
  [16, 8, 4],
  [12, 8, 4],
  [12, 8, 2],
  [8, 8, 8],
  [32, 16, 8],
  [24, 16, 8],
  [24, 16, 4]])